# VAR Fairness Audit: Data Extraction

**DS 112 Final Project**

This notebook focuses on loading and preparing the VAR datasets for analysis.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn plotly scikit-learn scipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)

## Data Loading

First, we'll load the VAR incident and team stats datasets. We'll explore multiple loading methods and handle potential file access issues.

In [ ]:
# Define file paths
VAR_INCIDENTS_FILE = 'VAR_Incidents_Stats.csv'
TEAM_STATS_FILE = 'VAR_Team_Stats.csv'
OUTPUT_FILE = 'var_combined.csv'

# Function to safely load CSV data
def safe_load_csv(file_path, description):
    try:
        data = pd.read_csv(file_path)
        print(f"✅ Successfully loaded {description} data with shape: {data.shape}")
        return data
    except FileNotFoundError:
        print(f"❌ Error: Could not find {file_path}")
        print(f"Make sure the {description} file is in the current working directory.")
        return None
    except pd.errors.EmptyDataError:
        print(f"❌ Error: The file {file_path} is empty.")
        return None
    except pd.errors.ParserError:
        print(f"❌ Error: The file {file_path} could not be parsed as CSV.")
        return None
    except Exception as e:
        print(f"❌ Error loading {file_path}: {str(e)}")
        return None

# Load the datasets
var_incidents = safe_load_csv(VAR_INCIDENTS_FILE, "VAR incidents")
team_stats = safe_load_csv(TEAM_STATS_FILE, "team stats")

# Check if both datasets were loaded successfully
if var_incidents is not None and team_stats is not None:
    print("\nBoth datasets loaded successfully!")
else:
    print("\n⚠️ Warning: One or both datasets failed to load.")
    print("Some code below may not work properly.")

# Display basic information about the datasets
if var_incidents is not None:
    print("\nVAR Incidents Dataset Info:")
    print(f"- Rows: {var_incidents.shape[0]}")
    print(f"- Columns: {var_incidents.shape[1]}")
    print("\nVAR Incidents Dataset - First 5 rows:")
    display(var_incidents.head())

if team_stats is not None:
    print("\nTeam Stats Dataset Info:")
    print(f"- Rows: {team_stats.shape[0]}")
    print(f"- Columns: {team_stats.shape[1]}")
    print("\nTeam Stats Dataset - First 5 rows:")
    display(team_stats.head())

## Data Understanding

Let's examine the structure and content of both datasets in detail.

In [ ]:
# Examine the column names and data types
if var_incidents is not None:
    print("VAR Incidents Dataset - Column Information:")
    print(var_incidents.info())
    print("\nColumn Descriptions:")
    for col in var_incidents.columns:
        print(f"- {col}")

if team_stats is not None:
    print("\nTeam Stats Dataset - Column Information:")
    print(team_stats.info())
    print("\nColumn Descriptions:")
    for col in team_stats.columns:
        print(f"- {col}")

# Check for unique values in categorical columns (if VAR incidents exists)
if var_incidents is not None:
    print("\nUnique values in key VAR incident columns:")
    categorical_cols = ['team_name', 'decision_type', 'match_minute', 'season']
    
    for col in categorical_cols:
        if col in var_incidents.columns:
            print(f"\n{col} - {var_incidents[col].nunique()} unique values:")
            print(var_incidents[col].value_counts().head(10))

## Data Cleaning

Next, we'll clean the datasets by handling missing values, standardizing formats, and addressing any inconsistencies.

In [ ]:
# Function to analyze missing values
def analyze_missing_values(df, dataset_name):
    print(f"\n{dataset_name} - Missing Value Analysis:")
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_df = pd.DataFrame({'Missing Values': missing, 'Percentage': missing_percent})
    missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values('Missing Values', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df)
    else:
        print("No missing values found!")
    return missing_df

# Analyze missing values in both datasets
if var_incidents is not None:
    var_missing = analyze_missing_values(var_incidents, "VAR Incidents Dataset")

if team_stats is not None:
    team_missing = analyze_missing_values(team_stats, "Team Stats Dataset")

# Function to clean dataset
def clean_dataset(df, dataset_name):
    print(f"\nCleaning {dataset_name}...")
    # Make a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # 1. Handle missing values
    for col in cleaned_df.columns:
        missing_count = cleaned_df[col].isnull().sum()
        if missing_count > 0:
            print(f"- Column '{col}' has {missing_count} missing values")
            
            # Determine data type and handle accordingly
            if pd.api.types.is_numeric_dtype(cleaned_df[col]):
                # Fill numeric columns with median
                median_val = cleaned_df[col].median()
                cleaned_df[col].fillna(median_val, inplace=True)
                print(f"  ✓ Filled with median value: {median_val}")
            else:
                # Fill categorical columns with mode
                mode_val = cleaned_df[col].mode()[0]
                cleaned_df[col].fillna(mode_val, inplace=True)
                print(f"  ✓ Filled with most common value: {mode_val}")
    
    # 2. Standardize text fields (lowercase for text columns)
    for col in cleaned_df.columns:
        if cleaned_df[col].dtype == 'object':
            try:
                # Convert to lowercase if it's a string column
                cleaned_df[col] = cleaned_df[col].str.strip().str.lower()
                print(f"- Standardized text format in column '{col}'")
            except:
                print(f"- Could not standardize column '{col}'")
    
    # 3. Remove duplicate rows
    dupes = cleaned_df.duplicated().sum()
    if dupes > 0:
        print(f"- Found {dupes} duplicate rows")
        cleaned_df.drop_duplicates(inplace=True)
        print(f"  ✓ Removed all duplicate rows")
    else:
        print("- No duplicate rows found")
    
    print(f"✅ Cleaned {dataset_name} successfully!")
    return cleaned_df

# Clean the datasets
if var_incidents is not None:
    cleaned_incidents = clean_dataset(var_incidents, "VAR Incidents Dataset")
else:
    cleaned_incidents = None

if team_stats is not None:
    cleaned_team_stats = clean_dataset(team_stats, "Team Stats Dataset")
else:
    cleaned_team_stats = None

## Feature Engineering

Let's add some useful derived features before merging the datasets.

In [ ]:
# Feature engineering for VAR incidents
if cleaned_incidents is not None:
    print("Adding derived features to VAR incidents dataset...")
    
    # 1. Match time period (group match minutes into periods)
    if 'match_minute' in cleaned_incidents.columns:
        time_bins = [0, 15, 30, 45, 60, 75, 90, 120]
        time_labels = ['0-15', '16-30', '31-45', '46-60', '61-75', '76-90', '90+']
        cleaned_incidents['time_period'] = pd.cut(cleaned_incidents['match_minute'], 
                                                bins=time_bins, 
                                                labels=time_labels, 
                                                right=False)
        print("✓ Added 'time_period' based on match minute")
    
    # 2. Decision outcome - was it favorable to the team?
    if 'decision_type' in cleaned_incidents.columns:
        # Define favorable decisions (this is an assumption, adjust as needed)
        favorable_decisions = ['penalty_awarded', 'goal_allowed', 'red_card_to_opponent']
        unfavorable_decisions = ['penalty_overturned', 'goal_disallowed', 'red_card_to_team']
        
        # Create a decision outcome feature
        def determine_favorability(decision):
            if decision in favorable_decisions:
                return 1  # Favorable
            elif decision in unfavorable_decisions:
                return 0  # Unfavorable
            else:
                return 0.5  # Neutral
        
        cleaned_incidents['decision_favorable'] = cleaned_incidents['decision_type'].apply(determine_favorability)
        print("✓ Added 'decision_favorable' feature")
    
    # 3. Count decisions by team
    team_decision_counts = cleaned_incidents.groupby('team_name').size().reset_index(name='total_var_decisions')
    cleaned_incidents = pd.merge(cleaned_incidents, team_decision_counts, on='team_name', how='left')
    print("✓ Added 'total_var_decisions' by team")
    
    # Display the new features
    print("\nVAR Incidents with new features:")
    display(cleaned_incidents.head())

# Feature engineering for team stats
if cleaned_team_stats is not None:
    print("\nAdding derived features to team stats dataset...")
    
    # 1. Team tier based on ranking (if rank column exists)
    rank_column = next((col for col in cleaned_team_stats.columns if 'rank' in col.lower()), None)
    if rank_column:
        # Create team tiers (quartiles)
        cleaned_team_stats['team_tier'] = pd.qcut(cleaned_team_stats[rank_column], 
                                                q=4, 
                                                labels=['Top Tier', 'Upper Mid', 'Lower Mid', 'Bottom Tier'])
        print(f"✓ Added 'team_tier' based on {rank_column}")
    
    # Display the new features
    print("\nTeam Stats with new features:")
    display(cleaned_team_stats.head())

## Data Merging

Now we'll merge the two datasets to create a comprehensive dataset for analysis.

In [ ]:
# Merge the cleaned datasets
if cleaned_incidents is not None and cleaned_team_stats is not None:
    print("Merging VAR incidents and team stats datasets...")
    
    # Identify the common key for merging
    team_name_col_incidents = 'team_name'
    team_name_col_stats = 'team_name'
    
    # Check if team names match between datasets
    incidents_teams = set(cleaned_incidents[team_name_col_incidents].unique())
    stats_teams = set(cleaned_team_stats[team_name_col_stats].unique())
    
    common_teams = incidents_teams.intersection(stats_teams)
    incidents_only = incidents_teams - stats_teams
    stats_only = stats_teams - incidents_teams
    
    print(f"Teams in both datasets: {len(common_teams)}")
    print(f"Teams only in incidents: {len(incidents_only)}")
    print(f"Teams only in stats: {len(stats_only)}")
    
    if len(incidents_only) > 0:
        print("\nTeams in incidents but not in stats:")
        print(list(incidents_only)[:5]), # Show only first 5 if many
    
    # Perform the merge
    merged_df = pd.merge(cleaned_incidents, cleaned_team_stats, 
                        left_on=team_name_col_incidents, 
                        right_on=team_name_col_stats, 
                        how='left')
    
    # Check for successful merge
    print(f"\nMerged dataset shape: {merged_df.shape}")
    print(f"Original incidents shape: {cleaned_incidents.shape}")
    
    # Count rows where team stats data is missing
    missing_stats = merged_df[merged_df.iloc[:, cleaned_incidents.shape[1]:].isnull().all(axis=1)].shape[0]
    print(f"Incidents without team stats: {missing_stats} ({missing_stats/len(merged_df)*100:.1f}%)")
    
    # Display the merged dataset
    print("\nMerged Dataset - First 5 rows:")
    display(merged_df.head())
    
    # Save the merged dataset
    merged_df.to_csv(OUTPUT_FILE, index=False)
    print(f"\n✅ Merged dataset saved to '{OUTPUT_FILE}'")
else:
    print("\n❌ Could not merge datasets because one or both datasets are missing.")

## Data Validation

Let's verify the quality of our merged dataset before proceeding to analysis.

In [ ]:
# Load the saved merged dataset to verify it was saved correctly
try:
    verified_df = pd.read_csv(OUTPUT_FILE)
    print(f"Successfully loaded the merged dataset from {OUTPUT_FILE}")
    print(f"Dataset shape: {verified_df.shape}")
    
    # Basic validation checks
    print("\nRunning validation checks...")
    
    # 1. Check for missing values
    missing = verified_df.isnull().sum()
    missing_cols = missing[missing > 0]
    if len(missing_cols) > 0:
        print("\nColumns with missing values:")
        print(missing_cols)
    else:
        print("✓ No missing values found")
    
    # 2. Check for duplicates
    dupes = verified_df.duplicated().sum()
    if dupes > 0:
        print(f"⚠️ Found {dupes} duplicate rows in the merged dataset")
    else:
        print("✓ No duplicate rows found")
    
    # 3. Check column data types
    print("\nColumn data types:")
    print(verified_df.dtypes)
    
    # 4. Basic statistics for numerical columns
    print("\nBasic statistics for numerical columns:")
    print(verified_df.describe())
    
    print("\n✅ Data validation complete!")
    print("The dataset is ready for exploratory analysis and modeling.")
    
except Exception as e:
    print(f"❌ Error validating the merged dataset: {str(e)}")